<a href="https://colab.research.google.com/github/jjcremer/NLP_Masters/blob/main/NLP202/nlp202_assignment1_1542248.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)

In [ ]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')
print(vars(train_data.examples[0]))

downloading aclImdb_v1.tar.gz


100%|██████████| 84.1M/84.1M [00:03<00:00, 21.9MB/s]


Number of training examples: 25000
Number of testing examples: 25000
{'text': ['Small', 'SPOILERS', 'alert', '!', '!', '!', '<', 'br', '/><br', '/>Good', 'movie', '...', 'VERY', 'good', 'movie', '.', 'And', 'I', "'m", 'surprised', 'to', 'say', 'that', 'myself', ',', 'because', 'I', "'m", 'not', 'a', 'big', 'fan', 'of', 'vampires', 'and', 'the', 'sound', 'of', 'the', 'director', "'s", 'name', 'Deran', 'Serafian', 'usually', 'means', 'bad', 'news', '.', 'Most', 'of', 'his', 'films', 'are', 'below', 'average', 'action', 'movies', 'like', 'Death', 'Warrant', 'and', 'Gunmen', '.', 'This', 'was', 'one', 'of', 'his', 'first', 'films', 'and', 'maybe', 'he', 'should', 'have', 'continued', 'making', 'horror', 'movies', 'instead', 'of', 'action', '.', 'This', 'movie', 'really', 'fascinated', 'me', '.', 'Good', 'accomplishment', ',', 'seeing', 'no', 'famous', 'actors', 'or', 'big', 'budget', 'was', 'involved', '.', 'It', 'really', 'is', 'the', 'story', 'that', 'keeps', 'you', 'focused', '.', 'Espe

In [ ]:
import random
import numpy as np
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_data, valid_data = train_data.split(random_state = random.seed(SEED))
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [ ]:
from torch.utils.data import DataLoader
from functools import partial

def custom_collate_func(batch, window_size, word_to_ix, label_to_ix):
  #print(batch)
  x, y = zip(*batch)

  def pad_window(sentence, window_size, pad_token='<pad>'):
    window = [pad_token] * window_size
    return window + sentence + window
  #x = [pad_window(s, window_size=window_size) for s in x]

  def convert_token_to_indices(sentence, word_to_ix):
    return [word_to_ix.get(token, word_to_ix['<unk>']) for token in sentence]

  x = [convert_token_to_indices(s, word_to_ix) for s in x]

  pad_token_ix = word_to_ix['<pad>']

  lengths = [len(s) for s in x]
  lengths = torch.LongTensor(lengths)

  #print(x)
  x = [torch.LongTensor(x_i) for x_i in x]
  #x = torch.LongTensor(x)
  #print('x is:', x)
  x_padded = nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=pad_token_ix)
  x_padded = x_padded.to('cuda')
  #print('padded x is: ', x_padded)
  #print(x_padded)

  #print(y)
  y = [label_to_ix.get(y_i) for y_i in y]
  #print(y)
  y = [y_i for y_i in y]
  y = torch.FloatTensor(y)
  #print(device)
  y = y.to(device)
  #print('is y saved to cuda?:', y.is_cuda)
  #y_padded = nn.utils.rnn.pad_sequence(y, batch_first=True, padding_value=0)

  return x_padded, y, lengths

In [ ]:
MAX_VOCAB_SIZE = 25_000
print(train_data)
TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")
print(TEXT.vocab.freqs.most_common(20))
print(TEXT.vocab.itos[:10])
print(LABEL.vocab.stoi)
print(device)
# Do not use BucketIterator in your implementation because you are required to implement the padding and masking yourself.
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits((train_data, valid_data, test_data), batch_size=1, device=device)

NameError: ignored

In [ ]:
print(LABEL.vocab.stoi.get('pos'))
#print(.voab)

In [ ]:
#data = list(zip(train_sentence, train_labels))
BATCH_SIZE = 13
shuffle = False
window_size = 2

print(type(TEXT))
collate_fn = partial(custom_collate_func, window_size=window_size, word_to_ix=TEXT.vocab.stoi, label_to_ix=LABEL.vocab.stoi)

def examples_to_zip(data):
  pairs = []
  for example in data.examples:
    pairs.append((example.text, example.label))
  return pairs

zipped_train_data = examples_to_zip(train_data)

data_loader = DataLoader(zipped_train_data, batch_size=BATCH_SIZE, shuffle=shuffle, collate_fn=collate_fn)

In [ ]:
import torch.nn as nn

class batchLR(nn.Module):
    def __init__(self, input_dim, embedding_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.fc = nn.Linear(embedding_dim, output_dim)
    def forward(self, text, lengths=[]):
        #text = text.numpy().tolist()
        #text = [i for i in text]
        #text = torch.tensor(text)
        #print(text)
        #print(text.shape)
        embedded = self.embedding(text).sum(1)
        #print(embedded.shape)
        return self.fc(embedded).squeeze()#.sum(-1)

In [ ]:
import torch.nn as nn

class LR(nn.Module):
    def __init__(self, input_dim, embedding_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.fc = nn.Linear(embedding_dim, output_dim)
    def forward(self, text):
        embedded = self.embedding(text).sum(0).squeeze()
        return self.fc(embedded)

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    def forward(self, text, lengths):
        #text = text.numpy().tolist()
        #text = [i for i in text]
        #text = torch.tensor(text)
        #print(text)
        #print('Input Lengths: ', lengths.size())
        #print('Device: ', text.is_cuda)
        embedded = self.embedding(text)#.sum(1)#.squeeze()
        #print('Generated Embeddings: ', embedded.size())
        embedded_pack = nn.utils.rnn.pack_padded_sequence(embedded, lengths, batch_first=True, enforce_sorted=False)
        #print('Packed Embeddings: ', embedded_pack.batch_sizes)
        #print('Packed Embeddings unsorted: ', embedded_pack.unsorted_indices)
        lstm_out, (ht, ct) = self.lstm(embedded_pack)
        #print(embedded.shape)
        # return self.fc(embedded)
        #print('LSTM Output: ', ht[-1])
        return self.fc(ht[-1])

In [ ]:
INPUT_DIM = len(TEXT.vocab)
#print(INPUT_DIM)
EMBEDDING_DIM = 100
HIDDEN_DIM = 100
OUTPUT_DIM = 1
LEARNING_RATE = 0.9e-3

#model = LSTM(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
model = batchLR(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

In [ ]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)

In [ ]:
criterion = nn.BCEWithLogitsLoss()

In [ ]:
model = model.to(device)
#print(next(model.parameters()).is_cuda)
#model_batch = model_batch.to(device)
criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
from tqdm import tqdm
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for instance in tqdm(iterator, desc="Training...", total=len(iterator)):
        
        optimizer.zero_grad()
                
        predictions = model(instance.text)
        #print(type(predictions))
        #print(type(instance.label))
        loss = criterion(predictions, instance.label)
        
        acc = binary_accuracy(predictions, instance.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
from tqdm import tqdm
def batch_train(model, loader, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch_input, batch_labels, batch_lengths in tqdm(loader, desc="Training...", total=len(loader)):
        
        optimizer.zero_grad()
        #print(batch_input)
        #batch_input = batch_input.view(batch_input.size(0), -1)

        #print('input post view/size', torch.tensor(batch_input))
        #batch_input.to('cuda')
        #print('What the hell?', batch_input.is_cuda)
        predictions = model(batch_input, batch_lengths)
        #print(len(batch_input))
        #print()
        #print(len(batch_labels))
        #print()
        #print('Predictions: ')
        #print(predictions)
        #print('Labels: ')
        #print(batch_labels)
        #print(torch.Tensor.new_tensor(predictions))
        #print('Huh:', batch_labels.size())
        #if (predictions.size(0) == 1):
        #  print(predictions.item())
        #  predictions[0] = predictions.item()
        loss = criterion(predictions, batch_labels)
        
        acc = binary_accuracy(predictions, batch_labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    
    return epoch_loss / len(loader), epoch_acc / len(loader)

In [ ]:
#bla = next(iter(data_loader))
#bla

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for instance in iterator:

            predictions = model(instance.text)
            
            loss = criterion(predictions, instance.label)
            
            acc = binary_accuracy(predictions, instance.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def batch_evaluate(model, loader, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch_inputs, batch_labels, batch_lengths in loader:

            predictions = model(batch_inputs, batch_lengths)
            
            loss = criterion(predictions, batch_labels)
            
            acc = binary_accuracy(predictions, batch_labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(loader), epoch_acc / len(loader)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 11

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    #train_loss, train_acc = train(model, test_iterator, optimizer, criterion)
    #valid_loss, valid_acc = evaluate(model, test_iterator, criterion)
    
    train_loss, train_acc = batch_train(model, data_loader, optimizer, criterion)
    valid_loss, valid_acc = batch_evaluate(model, data_loader, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

In [ ]:
o_model = LR(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM)
o_model.to(device)

o_model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = batch_evaluate(o_model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

In [ ]:
# Test of model correctness
max_n_test_instances = 5
i = 1
for instance in valid_iterator:
  score = model(instance.text)
  print(score)
  if i >= max_n_test_instances:
    break
  else:
    i += 1
